# Libraries

In [1]:
import os 
import numpy as np
import pandas as pd
import mysql.connector

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark Env") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

22/09/23 17:36:56 WARN Utils: Your hostname, cesar-GL62M-7RDX resolves to a loopback address: 127.0.1.1; using 10.22.162.210 instead (on interface wlp2s0)
22/09/23 17:36:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/23 17:36:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# User-Defined functions

In [2]:
# For privacy reasons, User Information is located in a .txt file in the same folder
# First row contains the user name
# Second row contains the password

try:
    login = pd.read_csv(r'../mysql_login.txt', header=None)
    mysql_user = login[0][0]
    mysql_pwd = login[0][1]
    print('User information is ready!')
except:
    print('Login information is NOT available!!!')

User information is ready!


In [3]:
# Establish a connection. Define parameters
def sql_conn():
    conn = mysql.connector.connect(
      host="localhost"
      , user = mysql_user
      , password = mysql_pwd
      , database='HDL_Project')
    return conn

In [4]:
def sql_rdd(sqlq, rdd = True):
    
    # -----------------------
    # Description:
    # Reads MySQL data and transforms into spark RDD object
    
    # Parameters:
    # * sqlq: SQL query.
    # * rdd: Boolean parameter defining transformation into Spark RDD object if True. Else returns a pandas df.
    # -----------------------

    mysql_conn = sql_conn()
    
    mysql_cursor = mysql_conn.cursor()

    # Create a pandas dataframe
    df = pd.read_sql(sqlq, con=mysql_conn)
    
    mysql_cursor.close()

    # Convert Pandas dataframe to spark DataFrame
    if rdd == True:
        df = spark.createDataFrame(df)

    return df

# Test
if False:
    sqlq = "SELECT * FROM cat_stations"
    df = sql_rdd(sqlq)
    df.show()

+------------+-------------+
|station_code| station_name|
+------------+-------------+
|          SE|   La Pastora|
|          NE|  San Nicolás|
|          CE|     Obispado|
|          NO|  San Bernabé|
|          SO|Sta. Catarina|
|         NO2|       García|
|         NTE|     Escobedo|
|         NE2|      Apodaca|
|         SE2|       Juárez|
|         SO2|    San Pedro|
|         SE3|    Cadereyta|
|         SUR|Pueblo Serena|
|        NTE2|  Universidad|
+------------+-------------+

